In [1]:
import os
import pandas as pd
from pprint import pprint

# Define the directory containing the CSV files
overall_dir = "../ablations" 
model_dirs = [ f.path for f in os.scandir(overall_dir) if f.is_dir() ]
dirs = []
for folder in model_dirs:
    dirs.extend([ f.path for f in os.scandir(folder) if f.is_dir() ])

pprint(dirs)



['../trend/meta-llama+Llama-3.1-8B/nlu-n_selected_blocks=5-block_select=bm25',
 '../trend/meta-llama+Llama-3.1-8B/banking77-retrieval-n_selected_blocks=200-block_select=bm25',
 '../trend/meta-llama+Llama-3.1-8B/nlu-retrieval-n_selected_blocks=100-block_select=bm25',
 '../trend/meta-llama+Llama-3.1-8B/nlu-n_selected_blocks=150-block_select=bm25',
 '../trend/meta-llama+Llama-3.1-8B/nlu-n_selected_blocks=15-block_select=all',
 '../trend/meta-llama+Llama-3.1-8B/banking77-n_selected_blocks=18-block_select=all',
 '../trend/meta-llama+Llama-3.1-8B/nlu-n_selected_blocks=200-block_select=bm25',
 '../trend/meta-llama+Llama-3.1-8B/nlu-retrieval-n_selected_blocks=250-block_select=bm25',
 '../trend/meta-llama+Llama-3.1-8B/banking77-retrieval-n_selected_blocks=150-block_select=bm25',
 '../trend/meta-llama+Llama-3.1-8B/nlu-n_selected_blocks=3-block_select=bm25',
 '../trend/meta-llama+Llama-3.1-8B/nlu-n_selected_blocks=3-block_select=all',
 '../trend/meta-llama+Llama-3.1-8B/banking77-n_selected_blocks

In [2]:
for filename in os.listdir(dirs[0]):
    if filename.endswith(".csv") and 'n_shots' in filename and "samples" not in filename:
        file_path = os.path.join(dirs[0], filename)
        break
    
df = pd.read_csv(file_path)
df

,Unnamed: 0,outputs,label_tokens,true_numeric_labels,true_label_present_in_prompt,correct,predicted_label_present_in_prompt,prompt_labels,prompt_example_indices,id,n_shots,run_number,nspw
0,5082,lists create or add,general quirky,27,True,False,False,"{45: 30, 25: 34, 27: 29, 59: 21, 58: 5, 7: 34,...","[5650, 12185, 6809, 1601, 14321, 4987, 17014, ...",5082,750,6,50
1,806,general quirky,general quirky,27,True,True,True,"{45: 30, 25: 34, 27: 29, 59: 21, 58: 5, 7: 34,...","[5650, 12185, 6809, 1601, 14321, 4987, 17014, ...",806,750,6,50
2,4902,calendar remove,calendar remove,8,True,True,True,"{45: 30, 25: 34, 27: 29, 59: 21, 58: 5, 7: 34,...","[5650, 12185, 6809, 1601, 14321, 4987, 17014, ...",4902,750,6,50
3,3855,general joke,general joke,24,True,True,True,"{45: 30, 25: 34, 27: 29, 59: 21, 58: 5, 7: 34,...","[5650, 12185, 6809, 1601, 14321, 4987, 17014, ...",3855,750,6,50
4,3380,weather query,calendar query,7,True,False,False,"{45: 30, 25: 34, 27: 29, 59: 21, 58: 5, 7: 34,...","[5650, 12185, 6809, 1601, 14321, 4987, 17014, ...",3380,750,6,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,5630,general repeat,general repeat,28,True,True,True,"{45: 30, 25: 34, 27: 29, 59: 21, 58: 5, 7: 34,...","[5650, 12185, 6809, 1601, 14321, 4987, 17014, ...",5630,750,6,50
246,5117,calendar query,calendar query,7,True,True,True,"{45: 30, 25: 34, 27: 29, 59: 21, 58: 5, 7: 34,...","[5650, 12185, 6809, 1601, 14321, 4987, 17014, ...",5117,750,6,50
247,2649,iot wemo on,iot wemo on,37,True,True,True,"{45: 30, 25: 34, 27: 29, 59: 21, 58: 5, 7: 34,...","[5650, 12185, 6809, 1601, 14321, 4987, 17014, ...",2649,750,6,50
248,752,email sendemail,email send,17,True,True,False,"{45: 30, 25: 34, 27: 29, 59: 21, 58: 5, 7: 34,...","[5650, 12185, 6809, 1601, 14321, 4987, 17014, ...",752,750,6,50


In [3]:
def get_labelset_info(df):
    all_output_labels = list(set(df['label_tokens']))
    all_labels = {k: {'TP': 0, 'FP': 0, 'FN': 0} for k in list(set(df['outputs']))}
    # need to track precision and recall for each class 
    def starts_any(prefix, longlist):
        for i in longlist:
            if i.startswith(prefix):
                return i
        return False

    for index, example in df.iterrows():
        if example['correct']:
            # this is a TP for one class 
            all_labels[example['outputs']]['TP'] = all_labels[example['outputs']]['TP'] + 1
        else:
            # this is a FP for the class we predicted...  
            all_labels[example['outputs']]['FP'] = all_labels[example['outputs']]['FP'] + 1       
            
            # ... and a FN for the true class
            if key := starts_any(example['label_tokens'], all_labels.keys()):
                all_labels[key]['FN'] = all_labels[key]['FN'] + 1
            else:
                #print(f"WARNING: creating new key for label {example['label_tokens']}")
                all_labels[example['label_tokens']] = {'TP': 0, 'FP': 0, 'FN': 1} 
    return all_labels

In [4]:
def calc_f1(df, labelsize):
    labelset = get_labelset_info(df)
    sum_f1s = 0
    for label in labelset:
        count = labelset[label]
        if count['TP'] == 0:
            p = r = f1 = 0
        else:
            p = count['TP']/(count['TP'] + count['FP'])
            r = count['TP']/(count['TP'] + count['FN'])
            f1 = 2* ((p*r)/(p+r))
        sum_f1s += f1
    #if len(labelset) not in [6, 50, 68, 77, 151]:
    #    print(f"warning: {len(labelset)} is unexpected value!")
    return sum_f1s/labelsize 
    
calc_f1(df, 77)

0.6015350059291928

In [5]:
pprint(os.listdir(dirs[0]))

['nspw=50_+n_shots=750+run=6.csv',
 'nspw=50_+n_shots=750+run=1.csv',
 'nspw=50_+n_shots=750+run=8.csv',
 'nspw=50_+n_shots=750+run=9.csv',
 'nspw=50_+n_shots=750+run=0.csv',
 'nspw=50_+n_shots=750+run=7.csv',
 'combined_results.csv',
 'nspw=50_n_shots_results_750.npy',
 'nspw=50_+n_shots=750+run=4.csv',
 'nspw=50_+n_shots=750+run=3.csv',
 'n_shots_results_over_250_samples_seed_42_ri=False.csv',
 'nspw=50_+n_shots=750+run=2.csv',
 'nspw=50_+n_shots=750+run=5.csv',
 'nspw=50-n_shots.txt']


In [6]:
from tqdm import tqdm

for directory in dirs:
    results = [] # Create an empty list to store results

    # Function to calculate the percentage of correct values
    def calculate_percentage(file_path):
        # Load the CSV file into a pandas DataFrame
        df = pd.read_csv(file_path)

        # Count the number of rows where 'correct' column is True
        correct_count = df['correct'].sum()
        
        # Calculate the percentage of correct values
        total_rows = len(df.index)
        #assert total_rows == 250, f"Caution: not the expected test set size, test set size is {total_rows} for file {file_path}"
        percentage_correct = (correct_count / total_rows) * 100
        if percentage_correct > 100:
            print(correct_count)
            print(total_rows)

        def get_labelsize(filename):
            half = 1
            if "half" in filename:
                half=2
                
            size = 0
            if "trecfine" in filename:
                size = 50 // half
            elif "trec" in filename:
                size = 6 // half
            elif "nlu" in filename:
                size = 68 // half
            elif "banking" in filename:
                size = 77 // half
            elif "clinic" in filename:
                size = 151 // half
            elif "tacred" in filename:
                size = 42
            else:
                assert False, filename
            return size

        return percentage_correct, calc_f1(df, labelsize=get_labelsize(file_path))

    # Iterate through each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".csv") and 'n_shots' in filename and "samples" not in filename:
            file_path = os.path.join(directory, filename)
            
            # Extracting information from filename
            parts = filename.split("_")
            if "nspw=" in parts[0]:
                nspw = parts[0][len("nspw="):]
                n_shots = parts[2].split("shots=")[1].split("+")[0]
                # if nspw != n_shots: #### changed
                #     #print(f"mismatch between {nspw} and {n_shots}, skipping")
                #     # continue
            else:
                nspw = parts[1].split('+')[0][6:]
            parts_remaining = parts[-1].split("+run=")
            # n_shots = nspw ### changed 
            if n_shots == '':
                print(parts)
                assert False
            if len(parts_remaining) < 2:
                assert False; continue
            run_num = parts_remaining[1].split(".csv")[0]
            
            
            # Calculate the percentage of correct values
            percentage_correct, f1 = calculate_percentage(file_path)
            
            # Append results to the list
            results.append([n_shots, percentage_correct, run_num, nspw, f1])

    # Create a DataFrame from the results list
    result_df = pd.DataFrame(results, columns=['n_shots', 'accuracy', 'run_num', 'nspw', 'f1'])
    output_filename = 'combined_results.csv'
    output_file_path = os.path.join(directory, output_filename)
    if "retrieval" in output_file_path:
        print(output_file_path)
    result_df.to_csv(output_file_path, index=False)


../trend/meta-llama+Llama-3.1-8B/banking77-retrieval-n_selected_blocks=200-block_select=bm25/combined_results.csv
../trend/meta-llama+Llama-3.1-8B/nlu-retrieval-n_selected_blocks=100-block_select=bm25/combined_results.csv
../trend/meta-llama+Llama-3.1-8B/nlu-retrieval-n_selected_blocks=250-block_select=bm25/combined_results.csv
../trend/meta-llama+Llama-3.1-8B/banking77-retrieval-n_selected_blocks=150-block_select=bm25/combined_results.csv
../trend/meta-llama+Llama-3.1-8B/banking77-retrieval-n_selected_blocks=100-block_select=bm25/combined_results.csv
../trend/meta-llama+Llama-3.1-8B/nlu-retrieval-n_selected_blocks=200-block_select=bm25/combined_results.csv
../trend/meta-llama+Llama-3.1-8B/nlu-retrieval-n_selected_blocks=150-block_select=bm25/combined_results.csv
../trend/meta-llama+Llama-3.1-8B/banking77-retrieval-n_selected_blocks=250-block_select=bm25/combined_results.csv
../trend/meta-llama+Llama-3.1-8B/banking77-retrieval-n_selected_blocks=300-block_select=bm25/combined_results.cs

In [7]:
!cat ../test/togethercomputer+LLaMA-2-7B-32K/banking77/combined_results.csv


cat: ../test/togethercomputer+LLaMA-2-7B-32K/banking77/combined_results.csv: No such file or directory


In [8]:
!cat ../reproduce-out/togethercomputer+LLaMA-2-7B-32K_examples_stride=50/banking77/combined_results.csv

n_shots,accuracy,run_num,nspw,f1
838,79.2,3,838,0.7156850364642572
838,77.60000000000001,4,838,0.7514354502131427
838,79.60000000000001,5,838,0.7386829980985822
838,81.6,2,838,0.7643459714888283
838,80.4,8,838,0.7473478181270387
838,79.60000000000001,1,838,0.7540905342853391
838,79.60000000000001,6,838,0.7213281667827119
838,78.4,7,838,0.7249327151924551
838,78.4,0,838,0.7243736061917877
838,76.4,9,838,0.7104741578767549
